# Module Test Template

## 1. Module & Test Description

This notebook is used for testing the concrete area aggregation functionality in *concrete.py*

### Imports
##### General Imports

In [1]:
import os, sys, pathlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy as sc
import shapely as sh

import math

##### Extend PYPATH to current folder:
This allows importing libraries from the same folder; <code>pathlib.Path().resolve()</code> returns the path of the current directory.

In [2]:
sys.path.extend([pathlib.Path().resolve()])

Import specific testing modules:

In [3]:
# import concrete

## Testing Inputs
Below are the properties of several concrete regions, which we'l use for testing subsequent functions.

In [4]:
# Test inputing concrete regions as a 0D (single value), numpy array and python list:
as_float = 10.0
as_int = 5
as_list = [[10, 1], [1, 10], [2, 5]]

as_ndarray = np.array([[10, 1], [1, 10], [2, 5]])
as_0darray = np.array(5)
as_1darray = np.array([5, 2, 3, 4])
as_2darray = np.copy(as_ndarray)

In [5]:
# Return the type
type(as_float), type(as_int), type(as_list), type(as_ndarray)

(float, int, list, numpy.ndarray)

### Test dimensionality of lists and arrays

In [6]:
# For lists use a cheat:
def get_list_dimensions(my_list):
    if type(my_list) == list:
        temp = np.array(my_list)
        return temp.shape
    else:
        return 'Not a list type'

In [7]:
get_list_dimensions(as_list)

(3, 2)

In [8]:
def is_list(my_list):
    return True if type(my_list) == list else False

In [9]:
is_list(as_list)

True

In [10]:
# Let's create a function that converts a python list to a numpy list

In [11]:
# Now let's make sure we have numpy arrays:
type(as_0darray), type(as_1darray), type(as_2darray), type(as_ndarray)

(numpy.ndarray, numpy.ndarray, numpy.ndarray, numpy.ndarray)

In [12]:
# Now let's look at the shape of several numpy arrays
as_0darray.shape, as_1darray.shape, as_2darray.shape, as_ndarray.shape

((), (4,), (3, 2), (3, 2))

In [13]:
# Create a function turn a 0D array into at least a 1D array
def to_numpy(my_0darray):
    """Creates a numpy array. If 0D array or single value, turns it into a 1D array. 
    Existing numpy arrays are passed through. Python lists are turned into same dimension numpy arrays."""
    if type(my_0darray) == np.ndarray and my_0darray.shape == ():
        return np.array([my_0darray])
    elif type(my_0darray) == int or type(my_0darray) == float:
        return np.array([my_0darray])
    else:
        return np.array(my_0darray)

<span style="color: tomato">**WARNING**:</span> Note that the conditional tests for the shorthand module name *np.ndarray* while type() returns *numpy.ndarray* 

In [14]:
to_numpy(as_0darray), to_numpy(as_1darray), to_numpy(as_ndarray), to_numpy(as_list)

(array([5]),
 array([5, 2, 3, 4]),
 array([[10,  1],
        [ 1, 10],
        [ 2,  5]]),
 array([[10,  1],
        [ 1, 10],
        [ 2,  5]]))

In [15]:
to_numpy(as_0darray).shape, to_numpy(as_1darray).shape, to_numpy(as_ndarray).shape, to_numpy(as_list).shape

((1,), (4,), (3, 2), (3, 2))

In [16]:
# Test if the array is 1D or higher:
len(to_numpy(as_0darray).shape), len(to_numpy(as_1darray).shape), len(to_numpy(as_2darray).shape), len(as_list)

(1, 1, 2, 3)

<span style="color: tomato">**WARNING**:</span> If you use len() on numpy.ndarray.shape, you get the dimensionality, however, if you use it directly on a python list, you get the size of the list, which is not the same as the list dimension. **FIRST** use to_numpy() to convert to a numpy.ndarray.

In [17]:
def get_max_dim(values):
    #if (type(values) == list) | (type(values) == np.ndarray):
    arr = to_numpy(values)
    return len(arr)

In [18]:
as_float = 10.0
as_int = 5
as_list = [[10, 1], [1, 10], [2, 5]]

as_0darray = np.array(5)
as_1darray = np.array([5, 2, 3, 4])
as_ndarray = np.array([[10, 1], [1, 10], [2, 5]])

get_max_dim(as_ndarray)

3

<span style="color: orange">**TODO**:</span> There is an *issue* here with column or row vectors and which direction the data is structured...

## Rescale a Value with a New Range
Create function that rescales a value that originally lies within a given range to new range. By *default* the new range will be 0 -> 1. If <code>is_strict == True</code>, the function returns the new range min or max value, when the rescaled value occurs out of bounds.

In [19]:
def rescale_value(value, range_min, range_max,
                  scale_min = 0.0, scale_max = 1.0, 
                  is_strictly_bounded: bool = True):
    rescaled_value = (value - range_min) / (range_max - range_min) * (scale_max - scale_min) + scale_min 
    if is_strictly_bounded == True:
        rescaled_value = min(scale_max, max(scale_min, rescaled_value))
    return rescaled_value

In [20]:
# Input values
value = 0.5
range_min = 0
range_max = 1

scale_min = 1
scale_max = 3

is_strictly_bounded = True

In [21]:
# Test function above by changing inputs above
rescaled_value = rescale_value(value, range_min, range_max, scale_min, scale_max, is_strictly_bounded)
rescaled_value

2.0

In [22]:
# Input values
value = 5.422
range_min = 4.324
range_max = 6.432

# Test with default values
new_rescaled_value = rescale_value(value, range_min, range_max)
new_rescaled_value

0.5208728652751421

When determining the area of concrete to a depth of "c", the total area is the product of this function (<code>is_strictly_bounded == True</code>) and the area of each individual region. We'll use this function with the *strict bounds* imposed when calculating the net concrete area (see discussion below)

## Concrete Regions
We need a way to input a single width and height (i.e., a single concrete region), or multiple regions. I propose that we require either a list or numpy array for single or multiple region values.

In [23]:
# Create a region list and region list as numpy array (most desired)
regions_as_numpyarray = np.array([[10, 1], [1, 10], [2, 5]])
regions_as_list = [[10, 1], [1, 10], [2, 5]]

In [24]:
regions_as_numpyarray.shape, get_list_dimensions(regions_as_list)

((3, 2), (3, 2))

In [25]:
regions_as_list = to_numpy(regions_as_list)
regions_as_list, type(regions_as_list) # Not a list anymore...

(array([[10,  1],
        [ 1, 10],
        [ 2,  5]]),
 numpy.ndarray)

In [26]:
# Now, recreate the variables and let's start looking at some important functions
concrete_regions = np.array([[10, 1], [1, 10], [2, 5]])
concrete_regions_list = [[10, 1], [1, 10], [2, 5]]

In [27]:
# Create a function to determine the number of regions

def region_count(concrete_regions):
    return to_numpy(concrete_regions).shape[0]

In [28]:
region_count(concrete_regions), region_count(concrete_regions_list)

(3, 3)

## Concrete Region Heights
Need a function to take the individual region heights and create a "distance to region boundary" array. Also if <code>get_max == True</code> then it returns the *max* or *total* height of the section. **NOTE** by default <code>c == math.inf</code>, which means the max height will be returned if <code>get_max == True</code>, otherwise with other values of c, the effective height would be returned.

By default (<code>lower_bound = False</code>) means that the returned *heights* array gives the upper_bound height values.

In [29]:
def get_region_heights(concrete_regions):
    concrete_regions = to_numpy(concrete_regions)
    count = region_count(concrete_regions)
    heights = np.zeros(count)
    running_total = 0
    for i in range(count):
        running_total += concrete_regions[i][1]
        heights[i] = running_total
    return heights

In [30]:
def get_max_height(concrete_regions):
    return max(get_region_heights(concrete_regions))

In [31]:
def get_lower_bound_region_heights(concrete_regions):
    concrete_regions = to_numpy(concrete_regions)
    count = region_count(concrete_regions)
    heights = np.zeros(count)
    running_total = 0
    for i in range(1, count):
        running_total += concrete_regions[i-1][1]
        heights[i] = running_total
    return heights

In [32]:
heights = get_region_heights(concrete_regions)
heights

array([ 1., 11., 16.])

In [33]:
total_height = get_max_height(concrete_regions)
total_height

16.0

In [34]:
lbound_heights = get_lower_bound_region_heights(concrete_regions)
lbound_heights

array([ 0.,  1., 11.])

Need to modify the above function to accept different c values. Maybe we can use **rescale_value()** to assist in the process

In [35]:
def get_region(c, heights):
    region = 0
    for i in range(heights.shape[0]):
        if c <= heights[i]:
            region = i
            break
        elif c >= max(heights):
            region = heights.shape[0] - 1
    return region

**TEST** Create some test values for "c" and test out *get_regions()*

In [36]:
cs = [-math.inf, -1, 0, 0.25, 1, 1.23, 4, 10.5, 11, 11.5, 15, 16, 10000, math.inf]

print("n".rjust(2, " "), "reg".rjust(4," "),"val".rjust(8," "), "ub".rjust(6, " "), "lb".rjust(6, " "))
print("-"*30)
for i in range(len(cs)):
    print(str(i).rjust(2," "), 
          str(get_region(cs[i], heights)).rjust(4," "), 
          str(cs[i]).rjust(8," "), 
          str(heights[get_region(cs[i], heights)]).rjust(6," "),
          str(lbound_heights[get_region(cs[i], heights)]).rjust(6," "))

 n  reg      val     ub     lb
------------------------------
 0    0     -inf    1.0    0.0
 1    0       -1    1.0    0.0
 2    0        0    1.0    0.0
 3    0     0.25    1.0    0.0
 4    0        1    1.0    0.0
 5    1     1.23   11.0    1.0
 6    1        4   11.0    1.0
 7    1     10.5   11.0    1.0
 8    1       11   11.0    1.0
 9    2     11.5   16.0   11.0
10    2       15   16.0   11.0
11    2       16   16.0   11.0
12    2    10000   16.0   11.0
13    2      inf   16.0   11.0


<span style="color: dodgerblue">**NOTE**:</span>
1. Regions are counted from zero
2. The *get_region()* function returns *region 0* for values that are out of bounds (c < 0) and returns *region n* for values out of bounds beyond the max_height.
3. The upper bound height of a region is inclusive in that region, and exclusive (as lower-bound) in the next region

In [37]:
# Create a function to get the distance relative to the lower-bound of a region. Make sure the result is minimum 0.
# -----------------------------------------------------------------------------------------------------------------
# 1. Determine the region
# 2. Get the lower bound value for that region
# 3. Subtract c from the lower bound
# 4. Return the max of 0 or the subtraction
# 5. Make sure positive infinity doesn't blow it out

In [38]:
c = math.inf
region = get_region(c, get_region_heights(concrete_regions))
lower_bound = get_lower_bound_region_heights(concrete_regions)[region]
min(max(0, c - lower_bound), get_max_height(concrete_regions))

16.0

In [39]:
def get_distance_into_region(c, concrete_regions):
    region = get_region(c, get_region_heights(concrete_regions))
    lower_bound = get_lower_bound_region_heights(concrete_regions)[region]
    return min(max(0, c - lower_bound), get_max_height(concrete_regions))

In [40]:
# Create some new values for testing
new_c = math.inf
new_concrete_regions = [[10,1],[7,2],[1,11],[8,2.5]]
to_numpy(new_concrete_regions)

array([[10. ,  1. ],
       [ 7. ,  2. ],
       [ 1. , 11. ],
       [ 8. ,  2.5]])

In [41]:
# Take a look at the heights array:
get_region_heights(new_concrete_regions)

array([ 1. ,  3. , 14. , 16.5])

In [42]:
dist = get_distance_into_region(new_c, new_concrete_regions)
dist

16.5

<span style="color: dodgerblue">**NOTE**:</span> we can probably combine two functions by putting the default c value to *math.inf* to get the max height of the section


<span style="color: orange">**TODO**:</span> Clean up or combine functions.

## Concrete Region Areas

Create a function to return the areas of each region as a numpy array.

In [43]:
def region_areas(concrete_regions, c = math.inf, get_total: bool = False):
    concrete_regions = to_numpy(concrete_regions)
    count = region_count(concrete_regions)
    areas = np.zeros(count)
    for i in range(0, count):
        areas[i] = concrete_regions[i][0] * concrete_regions[i][1]
    return areas if get_total == False else sum(areas)

In [44]:
# Create new array of regions for testing
concrete_regions = [[8, 1], [1, 6], [7, 1]]

In [45]:
# Return a numpy array of the areas of each region
areas = region_areas(concrete_regions)
areas

array([8., 6., 7.])

In [46]:
# This time return the total area of all the regions using the same function

total_area = region_areas(concrete_regions, True)
total_area

array([8., 6., 7.])

### Create function to return array with lower and upper bounds of each region

In [47]:
def get_region_height_bounds(concrete_regions):
    concrete_regions = to_numpy(concrete_regions)
    count = region_count(concrete_regions)
    heights = np.zeros((count, 2))
    running_total = 0
    for i in range(count):
        heights[i,0] = running_total
        running_total += concrete_regions[i][1]
        heights[i,1] = running_total
    return heights

In [48]:
region_bounds = get_region_height_bounds(concrete_regions)
region_bounds

array([[0., 1.],
       [1., 7.],
       [7., 8.]])

In [49]:
c = 4
areas = region_areas(concrete_regions)

In [50]:
# Review how deep "c" goes into each region, then calculate the net area using rescale
for i in range(region_bounds.shape[0]):
    print(str(rescale_value(c, region_bounds[i][0], region_bounds[i][1])).rjust(4," "), 
          str(rescale_value(c, region_bounds[i][0], region_bounds[i][1])*areas[i]).rjust(5, " ")) 

 1.0   8.0
 0.5   3.0
 0.0   0.0


In [51]:
# Revise the original function to calculate net area:

def region_areas_net(concrete_regions, c = math.inf, get_total: bool = False):
    concrete_regions = to_numpy(concrete_regions)
    region_bounds = get_region_height_bounds(concrete_regions)
    count = concrete_regions.shape[0]
    areas = np.zeros(count)
    for i in range(0, count):
        areas[i] = concrete_regions[i][0] * concrete_regions[i][1] * rescale_value(c, region_bounds[i][0], region_bounds[i][1])
    return areas if get_total == False else sum(areas)

In [52]:
c = 4
new_areas = region_areas_net(concrete_regions, c, get_total = False)
new_areas

array([8., 3., 0.])

In [53]:
total_new_area = region_areas_net(concrete_regions, c, get_total = True)
total_new_area

11.0

## Get Centroid of Net Area
We need a function that will provide the centroid of the net-section (i.e., when c < max_height)

In [54]:
concrete_regions = [[10, 1],[1, 5],[10, 1]]

In [55]:
def get_region_thicknesses(concrete_regions):
    concrete_regions = to_numpy(concrete_regions)
    return concrete_regions[:, 1]

In [56]:
thicknesses = get_region_thicknesses(concrete_regions)
thicknesses, thicknesses.shape[0], len(thicknesses)

(array([1, 5, 1]), 3, 3)

In [57]:
c = math.inf
areas = region_areas_net(concrete_regions, c, get_total = False)
areas

array([10.,  5., 10.])

In [58]:
lower_bounds = get_lower_bound_region_heights(concrete_regions)
lower_bounds

array([0., 1., 6.])

In [59]:
moment = 0
area = 0
for i in range(thicknesses.shape[0]):
    dist_to_cg = lower_bounds[i] + thicknesses[i] * 0.5
    area += areas[i]
    moment += dist_to_cg * areas[i]
    print(i, dist_to_cg, areas[i], moment)
if area == 0:
    distance = 0
else:
    distance = moment / area
distance

0 0.5 10.0 5.0
1 3.5 5.0 22.5
2 6.5 10.0 87.5


3.5

In [60]:
# Create function; default c = math.inf for full section centroid.

In [61]:
def get_distance_to_net_centroid(concrete_regions, c = math.inf):
    concrete_regions = to_numpy(concrete_regions)
    thicknesses = get_region_thicknesses(concrete_regions)
    areas = region_areas_net(concrete_regions, c, get_total = False)
    lower_bounds = get_lower_bound_region_heights(concrete_regions)
    moment = 0
    area = 0
    for i in range(thicknesses.shape[0]):
        dist_to_cg = lower_bounds[i] + thicknesses[i] * 0.5
        area += areas[i]
        moment += dist_to_cg * areas[i]
    if area == 0:
        return 0
    else:
        return moment / area

In [63]:
dist_to_cg = get_distance_to_net_centroid(concrete_regions, c)
dist_to_cg

3.5

## <span style="color: orange">**TODO**:</span>

1. Verify centroid functionality on ConcreteSection
2. Net moment of inertia - is it necessary
4. Impliment with ConcreteSection